# Gradio

https://www.gradio.app/guides/quickstart - быстрый старт

https://www.gradio.app/playground - набор интерактивный примеров 

В 2023 году API пакета изменился, поэтому многие старые примеры не работают. Рекомендуется обращаться к документации или к новым статьям.


####  Hello, World!

Принцип построения программы

<img src="https://github.com/gradio-app/gradio/blob/main/guides/assets/dataflow.svg?raw=true" width=1150>

In [5]:
import gradio

# обработчик формы
# параметры - это данные полей (или др. элементы ввода) на форме
def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

# объект demo - форма
demo = gradio.Interface(
    fn=greet,                       # обработчик
    inputs=["text", "slider"],      # элементы ввода (поле ввода и слайдер)
    outputs=["text"],               # элементы вывода
    theme=gradio.themes.Base()
)
# https://www.gradio.app/guides/theming-guide   - настройка темы
    # gr.themes.Base()
    # gr.themes.Default()
    # gr.themes.Glass()
    # gr.themes.Monochrome()
    # gr.themes.Soft()


demo.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


## Некоторые элементы интерфейса

- `"text"`, `gradio.Textbox(···)`
- `"number"`, `gradio.Number(···)`
- `gradio.Label(···)`
- `gradio.Checkbox(···)`
- `gradio.Dropdown(···)`
- `gradio.Image(···)`
- `gradio.ImageEditor(···)` - примитивный редактор изображений, на котором можно (в том числе) рисовать
- ``
- ``
- ``
- ``

### Пример - вывод изображения

In [53]:
# подготовка данных для примеры
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn
import pandas as pd

data = load_iris()
X, y = data['data'], data['target']
TargetNames = data['target_names']

Data = pd.DataFrame(X)
Data.columns = data.feature_names
Data['class'] = y
Data['label'] = Data['class'].map({0:'setosa', 1:'versicolor', 2:'virginica'})

Forest = RandomForestClassifier().fit(X,y)

In [54]:
Data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class,label
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,virginica
146,6.3,2.5,5.0,1.9,2,virginica
147,6.5,3.0,5.2,2.0,2,virginica
148,6.2,3.4,5.4,2.3,2,virginica


In [66]:
def make_scatterplot(Data:pd.DataFrame, new_sw:float, new_pw: float):
    """Строит диаграмму рассеяния, добавляет новую точку в виде крестика"""
    fig = plt.figure( figsize=(5,5) )
    seaborn.scatterplot( data=Data[['sepal width (cm)', 'petal width (cm)', 'label']], x='sepal width (cm)', y='petal width (cm)', hue='label')
    # добавить ещё один маркер: размером 50, красного цвета
    plt.scatter(x=new_sw, y=new_pw, s=50, marker='x', color='r')
    # fig.show()
    plt.savefig('plot.png')
    return plt.imread('plot.png')

In [71]:
# features = sepal length (cm)     sepal width (cm)     petal length (cm)     petal width (cm)


def predict(sl, sw, pl, pw: float):
    """Обработчик для события отправки данных формы;
    параметры - данные формы"""
    global Forest, Data

    y_pred = Forest.predict([[sl, sw, pl, pw]])[0]     # модель sklearn принимает список объектов, поэтом тут второй список
    return make_scatterplot(Data, sw, pw), TargetNames[y_pred]
    # return должен соответствовать выходным элементам (outputs) на форме


seaborn.set_style("darkgrid")

iris_form2 = gradio.Interface(
    predict,
    inputs= [ gradio.Number(label="длина чашелистика (sepal length)"),  gradio.Number(label="ширина чашелистика (sepal width)"), 
              gradio.Number(label="длина лепестка (petal length)"),     gradio.Number(label="ширина лепестка (petal width)")],
    outputs=[gradio.Image(height=400, width=400), "text"]
)

iris_form2.launch()

# https://gradio.app/docs/#i_textbox

Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.
